In [230]:
import shutil
import time
import matplotlib.pyplot as plt
#import nltk
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
#from nltk.corpus import stopwords
from pandas_profiling import ProfileReport
from pyspark import SparkConf
from pyspark.ml import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.sql import Row, SparkSession
from pyspark.sql import Window 
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col,
    desc,
    expr,
    first,
    length,
    lit,
    regexp_replace,
    row_number,
    split,
    substring,
    to_date,
    to_timestamp,
    when,
)
from pyspark.sql.types import *
#from wordcloud import WordCloud
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import HashingTF, IDF,CountVectorizer
from pyspark.ml.clustering import BisectingKMeans

In [3]:
spark = SparkSession.builder.config("spark.driver.memory", "15g").config("spark.executor.memory", "15g")\
.config("spark.driver.extraJavaOptions","-Xss4M").master("local[*]").getOrCreate()


# lectura datos

## client_df

In [102]:
client_df = spark.read.json("yelp_academic_dataset_user.json")#.limit(10)
#client_df.pandas_api().head()#show(1)

In [105]:
client_df.write.mode("overwrite").parquet("yelp_academic_dataset_user2.parquet")

In [106]:
client_df = spark.read.parquet("yelp_academic_dataset_user2.parquet").dropDuplicates(["user_id"]).alias("client_df")
client_df.count()

1987897

In [84]:
client_df.select(F.max("user_id_numeric")).show()

+--------------------+
|max(user_id_numeric)|
+--------------------+
|             1987897|
+--------------------+



In [107]:
client_df.createOrReplaceTempView('df')
client_df = spark.sql('select row_number() over (order by "user_id") as user_id_numeric, * from df')
client_df.show()

+---------------+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+--------------------+-----+-------------+------------+------+--------------------+-------------------+
|user_id_numeric|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|elite|fans|             friends|funny|         name|review_count|useful|             user_id|      yelping_since|
+---------------+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+--------------------+-----+-------------+------------+------+--------------------+-------------------

In [108]:
client_df.write.mode("overwrite").parquet("yelp_academic_dataset_user.parquet")

In [ ]:
client_df = client_df.join(reviews_users_ids,client_df.user_id==reviews_users_ids.user_id)\
.drop(reviews_users_ids.user_id)
client_df.show()

In [5]:
user_df = spark.read.parquet("yelp_academic_dataset_user.parquet").alias("user_df")
user_df.count()

1987897

In [101]:
user_df.printSchema()

root
 |-- user_id_numeric: integer (nullable = true)
 |-- business_id_numeric: integer (nullable = true)
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_si

## Review_df

In [21]:
reviews_df = spark.read.json("yelp_academic_dataset_review.json").alias("reviews_df")#.limit(40000)

#reviews_df.pandas_api().head()#.show(1)

In [22]:
reviews_df.dropna().describe().show()

+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+----------------------+------------------+--------------------+
|summary|         business_id|              cool|               date|              funny|           review_id|             stars|                  text|            useful|             user_id|
+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+----------------------+------------------+--------------------+
|  count|             6990280|           6990280|            6990280|            6990280|             6990280|           6990280|               6990280|           6990280|             6990280|
|   mean|                null|0.4986175088837643|               null|0.32655959417934616|                null|  3.74858374771826|                  null|1.1846089140921394|                null|
| stddev|                null|2.172

In [23]:
reviews_df.select([F.count(when(F.isnan(c) | col(c).isNull(), c)).alias(c) for c in reviews_df.columns]).show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



In [24]:
reviews_df.select([F.count(when(col(c)=="null", c)).alias(c) for c in reviews_df.columns]).show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



In [25]:
reviews_df.write.mode("overwrite").parquet("yelp_academic_dataset_review.parquet")

In [109]:
reviews_df = spark.read.parquet("yelp_academic_dataset_review.parquet")\
.select("business_id","stars","user_id").alias("reviews_df").limit(20000)
reviews_df.count()

20000

In [10]:
reviews_df.show()

+--------------------+-----+--------------------+
|         business_id|stars|             user_id|
+--------------------+-----+--------------------+
|znK6tMeegKf9YnGut...|  5.0|TjD2S66AWxlKsPSQB...|
|UxwpCVLgPWCeaRyet...|  5.0|Vtfxxq3nxdYHRiH6O...|
|eVI64EQymywsvMLmD...|  5.0|ZDw_qN5Fy6PE0gdbV...|
|SPcPJfPgWzhjUDqVF...|  5.0|UvMDlX2wV4Md9OwGW...|
|T5XzQ6YnVExvd0BOR...|  5.0|ir9ixBZPzBwWe9IZ_...|
|9Ldo7ocJHVrEzRXUp...|  5.0|6WW9sM9H2pKMr01tw...|
|mdNm8X3vcedYXQ-vw...|  5.0|SVtNqAQFwOca_3yr1...|
|QVGHZ_gV-0fO8GeHn...|  5.0|pMj9HULhkLzg2qqcl...|
|sv3mfbZEI8frLbUbR...|  1.0|LVGZiLxiJe3VsUpaU...|
|0b7cBRywLcoqUpa3P...|  5.0|PPhQehweJHbMSTEp7...|
|WM89sIfap8g0JXmwX...|  4.0|R9IJ1Byr27n6wXj3Q...|
|7Iv-6B0EH-yVo5o_V...|  5.0|7JiwGIYyr8W4xguSs...|
|u7cZUEGK6iJP2gvI2...|  4.0|FUUEU3ZN5cL1nNNR2...|
|GWH_fPOdtIjFZT4-X...|  1.0|LY_YVPuJogpjX2MSa...|
|wkKhsrQe4CQAmYW7I...|  1.0|K4bFNYdR23y9PXnGg...|
|L_HVYZJEiHXaMJjG3...|  5.0|PFX0lEkTPyBS_dBbu...|
|lm2ZtQMo_CjTI4LFc...|  1.0|LC3fpdaTtp7iszHel...|


In [4]:
reviews_bussines_ids = reviews_df.select('business_id').distinct()\
.withColumn("business_id_numeric",F.monotonically_increasing_id()).alias("reviews_bussines_ids")
reviews_bussines_ids.show(5)

+--------------------+-------------------+
|         business_id|business_id_numeric|
+--------------------+-------------------+
|znK6tMeegKf9YnGut...|                  0|
|UxwpCVLgPWCeaRyet...|                  1|
|eVI64EQymywsvMLmD...|                  2|
|SPcPJfPgWzhjUDqVF...|                  3|
|T5XzQ6YnVExvd0BOR...|                  4|
+--------------------+-------------------+
only showing top 5 rows



In [5]:
reviews_bussines_ids.count()

150346

In [8]:
reviews_df.count()

6990280

In [7]:
reviews_users_ids.count()

1987929

In [12]:
reviews_users_ids = reviews_df.select('user_id').distinct()\
.withColumn("user_id_numeric",F.monotonically_increasing_id()).alias("reviews_users_ids")
#reviews_users_ids.show()

### Review corregido

In [11]:
reviews_df = spark.read.parquet("yelp_academic_dataset_review.parquet")\
.select("business_id","stars","user_id").alias("reviews_df")#.limit(1000000)
reviews_df.count()

6990280

In [12]:
join_1 = reviews_df.join(business_df.select("business_id","business_id_numeric"),business_df.business_id==reviews_df.business_id).drop(reviews_df.business_id)
join_1.show(1)

+-----+--------------------+--------------------+-------------------+
|stars|             user_id|         business_id|business_id_numeric|
+-----+--------------------+--------------------+-------------------+
|  5.0|TjD2S66AWxlKsPSQB...|znK6tMeegKf9YnGut...|              41686|
+-----+--------------------+--------------------+-------------------+
only showing top 1 row



In [13]:
reviews_final_df = join_1.join(user_df.select("user_id","user_id_numeric"),join_1.user_id==user_df.user_id).drop(join_1.user_id).alias("reviews_final_df").cache()#.limit(10000).cache()#.cache()#.drop("reviews_users_ids.user_id")
reviews_final_df.show(5)

+-----+--------------------+-------------------+--------------------+---------------+
|stars|         business_id|business_id_numeric|             user_id|user_id_numeric|
+-----+--------------------+-------------------+--------------------+---------------+
|  1.0|aIoUwpy5ZFQXUDxWM...|              88517|--RJK834fiQXm21Vp...|              3|
|  5.0|K_s-9Wd6vXSfnxYFz...|             111107|--UhENQdbuWEh0mU5...|              4|
|  4.0|oD3zBLplcYefdMHo5...|              26590|--cxdcv_b9uhAKsKT...|              7|
|  5.0|P0BB_HeVN-M8D31yt...|              67027|-0EzgKMI9ZakqLiWR...|             13|
|  5.0|6aPXOXi8h1m58hihg...|              82320|-0EzgKMI9ZakqLiWR...|             13|
+-----+--------------------+-------------------+--------------------+---------------+
only showing top 5 rows



In [ ]:
import time
start_time = time.time()
print(reviews_final_df.show())

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))


## Bussines_df

In [55]:
business_df = spark.read.json("yelp_academic_dataset_business.json")#.limit(10)
#business_df.pandas_api().head()#.show(1)

In [36]:
reviews_bussines_ids = business_df.select('business_id').distinct()\
.withColumn("business_id_numeric",F.monotonically_increasing_id()).alias("reviews_bussines_ids")
reviews_bussines_ids.show(5)

+--------------------+-------------------+
|         business_id|business_id_numeric|
+--------------------+-------------------+
|WKMJwqnfZKsAae75R...|                  0|
|knQ4vIgx-r85kjlWV...|                  1|
|1dSKEitDDgIkaApe6...|                  2|
|adATTqggIQX5xxLDI...|                  3|
|vxrGNnuEef7YCfB7m...|                  4|
+--------------------+-------------------+
only showing top 5 rows



In [54]:
reviews_bussines_ids.describe().show()

+-------+--------------------+--------------------+
|summary|         business_id| business_id_numeric|
+-------+--------------------+--------------------+
|  count|              150346|              150346|
|   mean|                null|1.063863211089765E10|
| stddev|                null| 7.125266116504055E9|
|    min|---kPU91CF4Lq2-Wl...|                   0|
|    max|zzyx5x0Z7xXWWvWnZ...|         17179943110|
+-------+--------------------+--------------------+



In [ ]:
business_df = business_df.join(reviews_bussines_ids,business_df.business_id==reviews_bussines_ids.business_id)\
.drop(reviews_bussines_ids.business_id)
business_df.show()

In [70]:
business_df.write.mode("overwrite").parquet("yelp_academic_dataset_business.parquet")

In [71]:
business_df = spark.read.parquet("yelp_academic_dataset_business.parquet")\
.alias("business_df")
business_df.describe().show()

+-------+-------------------+-----------------+--------------------+--------------------+-----------+-------------------+-----------------+------------------+--------------------+-----------------+------------------+------------------+------+
|summary|business_id_numeric|          address|         business_id|          categories|       city|            is_open|         latitude|         longitude|                name|      postal_code|      review_count|             stars| state|
+-------+-------------------+-----------------+--------------------+--------------------+-----------+-------------------+-----------------+------------------+--------------------+-----------------+------------------+------------------+------+
|  count|             150346|           150346|              150346|              150243|     150346|             150346|           150346|            150346|              150346|           150346|            150346|            150346|150346|
|   mean|            75173.5

In [60]:
W = W.partitionBy(lit(1))

In [68]:
business_df.createOrReplaceTempView('df')
business_df = spark.sql('select row_number() over (order by "business_id") as business_id_numeric, * from df')


In [ ]:
business_df.show()

In [4]:
business_df = spark.read.parquet("yelp_academic_dataset_business.parquet").alias("business_df")
business_df.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|business_id_numeric|             address|          attributes|         business_id|          categories|        city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+-------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|                  1|       120 W Main St|{null, null, null...|xKoz9eM8hUELf5qix...|Home & Garden, Ho...|  Plainfield|{10:0-18:0, null,...|      1|    39.703082|    -86.4027716|African Plum Home...|      46168|           5|  4.5|   IN|
|                  2| 2224 E Fletcher Ave|{null, null, u

## Checkin_df

In [7]:
checkin_df = spark.read.json("data/yelp_dataset/yelp_academic_dataset_checkin.json").limit(10)
checkin_df.pandas_api().head()#.show(1)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


## tip_df

In [6]:
tip_df = spark.read.json("data/yelp_dataset/yelp_academic_dataset_tip.json").limit(10)
tip_df.pandas_api().head()#.show(1)

,business_id,compliment_count,date,text,user_id
0,3uLgwr0qeCNMjKenHJwPGQ,0,2012-05-18 02:17:21,Avengers time with the ladies.,AGNUgVwnZUey3gcPCJ76iw
1,QoezRbYQncpRqyrLH6Iqjg,0,2013-02-05 18:35:10,They have lots of good deserts and tasty cuban...,NBN4MgHP9D3cw--SnauTkA
2,MYoRNLb5chwjQe3c_k37Gg,0,2013-08-18 00:56:08,It's open even when you think it isn't,-copOvldyKh1qr-vzkDEvw
3,hV-bABTK-glh5wj31ps_Jw,0,2017-06-27 23:05:38,Very decent fried chicken,FjMQVZjSqY8syIO-53KFKw
4,_uN0OudeJ3Zl_tf6nxg5ww,0,2012-10-06 19:43:09,Appetizers.. platter special for lunch,ld0AperBXk1h6UbqmM80zw


# Collaboration model

In [14]:
reviews_final_df.printSchema()

root
 |-- stars: double (nullable = true)
 |-- business_id: string (nullable = true)
 |-- business_id_numeric: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_id_numeric: integer (nullable = true)



In [15]:


train_df, test_df = reviews_final_df.select("business_id_numeric","stars","user_id_numeric").randomSplit([0.8,0.2])


train_df = train_df.cache()
test_df = test_df.cache()

In [162]:
train_df.count()

5592721

In [164]:
test_df.count()

1397526

In [22]:
 # train ALS model
import time
from pyspark.ml.recommendation import ALS, ALSModel
start_time = time.time()

als = ALS(maxIter=5, implicitPrefs=False,nonnegative=True,userCol="user_id_numeric", itemCol="business_id_numeric", ratingCol="stars",coldStartStrategy="drop")
model = als.fit(train_df)    
model.write().overwrite().save('spark_ALS_model')
persistedALSModel = ALSModel.load("spark_ALS_model")
print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

#model.write().overwrite().save('/usr/src/app/spark_ALS_model')
#persistedALSModel = PipelineModel.load("/usr/src/app/spark_rf_model")


Total Runtime: 60.26 seconds


In [ ]:
2147483647
68719477203

In [49]:
model.getUserCol()


'user_id_numeric'

In [23]:
predictions = persistedALSModel.transform(test_df)
predictions.show()

+-------------------+-----+---------------+----------+
|business_id_numeric|stars|user_id_numeric|prediction|
+-------------------+-----+---------------+----------+
|             134989|  5.0|             53|  4.784931|
|             146619|  5.0|             53|  5.129341|
|              45969|  4.0|             78| 0.6580459|
|              59279|  4.0|            103| 4.3934727|
|             147082|  5.0|            108| 1.1513397|
|             140514|  5.0|            137| 3.8620214|
|             136568|  4.0|            192|  2.956036|
|              44930|  4.0|            253| 3.5839498|
|              46363|  5.0|            253| 3.3202004|
|             106598|  4.0|            253| 4.7191906|
|              63989|  5.0|            271| 3.7924955|
|              60514|  5.0|            296| 2.9828622|
|              60999|  5.0|            296| 2.4339318|
|              23992|  5.0|            321|  4.164449|
|               9907|  3.0|            385| 2.1610692|
|         

In [15]:
predictions.describe().show()

+-------+-------------------+------------------+-----------------+------------------+
|summary|business_id_numeric|             stars|  user_id_numeric|        prediction|
+-------+-------------------+------------------+-----------------+------------------+
|  count|            1141361|           1141361|          1141361|           1141361|
|   mean|  75276.65015450852| 3.797917573843858|990624.8083130579| 3.294412698556364|
| stddev|  43331.03219439372|1.4102227707717347|573472.2705220649|1.1531156480862572|
|    min|                  1|               1.0|                2|               0.0|
|    max|             150346|               5.0|          1987881|         11.684158|
+-------+-------------------+------------------+-----------------+------------------+



In [26]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

evaluator = RegressionEvaluator(metricName="r2", labelCol="stars",
                                predictionCol="prediction")
r2 = evaluator.evaluate(predictions)

evaluator = RegressionEvaluator(metricName="mse", labelCol="stars",
                                predictionCol="prediction")
mse = evaluator.evaluate(predictions)


print(rmse)
print(mse)
print(r2)

1.5432466395747944
2.381610190558895
-0.19882590791378085


In [20]:
w = Window.partitionBy("user_id_numeric")

In [26]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())
df = predictions
# Iterating over columns to be scaled
for i in ["prediction"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(min= 1, max = 5, inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df = pipeline.fit(df).transform(df).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

df.show()

+-------------------+-----+---------------+----------+-----------------+
|business_id_numeric|stars|user_id_numeric|prediction|prediction_Scaled|
+-------------------+-----+---------------+----------+-----------------+
|              99905|  5.0|             27| 2.1880224|            1.749|
|             134989|  5.0|             53| 4.6566916|            2.594|
|              56535|  1.0|             78| 2.6789527|            1.917|
|              99374|  1.0|            126|0.94163007|            1.322|
|              36105|  5.0|            159| 1.9328552|            1.662|
|              53722|  5.0|            159|  1.394835|            1.478|
|              39921|  2.0|            236| 1.5677186|            1.537|
|              73020|  3.0|            296| 3.1335728|            2.073|
|                706|  5.0|            321| 4.0231266|            2.377|
|               1329|  1.0|            385| 2.8808742|            1.986|
|              81619|  5.0|            406| 1.91070

In [34]:


evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
                                predictionCol="prediction_Scaled")
rmse = evaluator.evaluate(df)

evaluator = RegressionEvaluator(metricName="r2", labelCol="stars",
                                predictionCol="prediction_Scaled")
r2 = evaluator.evaluate(df)

evaluator = RegressionEvaluator(metricName="mse", labelCol="stars",
                                predictionCol="prediction_Scaled")
mse = evaluator.evaluate(df)


print(rmse)
print(mse)
print(r2)

2.1278384749288426
4.527696575387503
-1.2766813476197236


In [171]:
persistedALSModel.recommendForAllUsers(3).show()

+---------------+--------------------+
|user_id_numeric|     recommendations|
+---------------+--------------------+
|             26|[{127544, 1.58875...|
|             27|[{26180, 9.890019...|
|             28|[{34396, 8.543524...|
|             31|[{134405, 2.04932...|
|             44|[{78744, 9.650007...|
|             53|[{71793, 12.37235...|
|             65|[{114882, 8.34195...|
|             78|[{3537, 2.64466},...|
|             81|[{16273, 8.47099}...|
|             85|[{142737, 11.0975...|
|            101|[{28581, 8.835049...|
|            103|[{105118, 12.5472...|
|            108|[{44155, 5.549817...|
|            115|[{26113, 1.924355...|
|            126|[{44487, 3.027095...|
|            133|[{47437, 2.012877...|
|            137|[{17092, 11.36896...|
|            148|[{100615, 2.30769...|
|            155|[{75541, 10.48246...|
|            159|[{76213, 11.33722...|
+---------------+--------------------+
only showing top 20 rows



In [86]:
x = persistedALSModel.recommendForAllUsers(3).filter(col('user_id_numeric')==22).select("recommendations").collect()


type(x[0][0][0])


pyspark.sql.types.Row

In [ ]:
def recommendedArtists(userId,limit):
    test =  persistedALSModel.recommendForAllUsers(limit).filter(col('user_id_numeric')==userId).select("recommendations").collect()
    topRecommendations = []
    for item in test[0][0]:
        topRecommendations.append(item.business_id_numeric)
 
    schema = StructType([StructField("user_id_numeric",IntegerType(),True)])
    recommendation = spark.createDataFrame(topRecommendations,IntegerType())
    final=recommendation.join(business_df,recommendation.value==business_df.business_id_numeric).select(business_df.business_id,business_df.name)
    return final# display top 10 recommended for user X
recommendedArtists(22,3).show(truncate=False)

In [39]:
test_df.printSchema()

root
 |-- business_id_numeric: long (nullable = false)
 |-- stars: double (nullable = true)
 |-- user_id_numeric: long (nullable = false)



In [ ]:
valid_data = test_df.map(lambda p: (p[0], p[1]))
valid_data.show()

In [ ]:
 # make prediction
valid_data = test_df.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
# get the rating result
ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
# get the RMSE
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
error = math.sqrt(MSE)

In [16]:


def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            
           
            als = ALS(maxIter=num_iters,rank=rank,regParam=reg, implicitPrefs=False,\
                      userCol="user_id_numeric",nonnegative=True, itemCol="business_id_numeric", ratingCol="stars",coldStartStrategy="drop")
            model = als.fit(train_data)    # (userID, productID, rating) tuple)
            # make prediction
            
            predictions = model.transform(validation_data)
            # get the rating result
            evaluator = RegressionEvaluator(metricName="mse", labelCol="stars",
                                predictionCol="prediction")
            error = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model



In [19]:
import time

# hyper-param config
num_iterations = 10
ranks = [10,12,15]
reg_params = [0.1,0.2,0.01,0.15,0.3]

# grid search and select best model
start_time = time.time()
final_model = train_ALS(train_df, test_df, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))


10 latent factors and regularization = 0.1: validation RMSE is 2.2557690854019787
10 latent factors and regularization = 0.2: validation RMSE is 2.0128867092011666
10 latent factors and regularization = 0.01: validation RMSE is 4.114120350285798
10 latent factors and regularization = 0.15: validation RMSE is 2.1029457852150863
10 latent factors and regularization = 0.3: validation RMSE is 1.9333393101141803
12 latent factors and regularization = 0.1: validation RMSE is 2.2431595415209338
12 latent factors and regularization = 0.2: validation RMSE is 2.0111345286259223
12 latent factors and regularization = 0.01: validation RMSE is 3.984024407821581
12 latent factors and regularization = 0.15: validation RMSE is 2.0986403281875425
12 latent factors and regularization = 0.3: validation RMSE is 1.9328934571117027
15 latent factors and regularization = 0.1: validation RMSE is 2.226105961239465
15 latent factors and regularization = 0.2: validation RMSE is 2.0087077926835613
15 latent facto

In [20]:

# hyper-param config
num_iterations = 15
ranks = [10,12,15]
reg_params = [0.1,0.2,0.01,0.15,0.3]

# grid search and select best model
start_time = time.time()
final_model = train_ALS(train_df, test_df, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

10 latent factors and regularization = 0.1: validation RMSE is 2.205096938445347
10 latent factors and regularization = 0.2: validation RMSE is 1.9917038394270494
10 latent factors and regularization = 0.01: validation RMSE is 3.518367515215485
10 latent factors and regularization = 0.15: validation RMSE is 2.0680185091615213
10 latent factors and regularization = 0.3: validation RMSE is 1.9294583203459708
12 latent factors and regularization = 0.1: validation RMSE is 2.1913540852647464
12 latent factors and regularization = 0.2: validation RMSE is 1.9888842839640448
12 latent factors and regularization = 0.01: validation RMSE is 3.4082580408187284
12 latent factors and regularization = 0.15: validation RMSE is 2.0625524506687833
12 latent factors and regularization = 0.3: validation RMSE is 1.9278572059611998
15 latent factors and regularization = 0.1: validation RMSE is 2.1753425679812226
15 latent factors and regularization = 0.2: validation RMSE is 1.9855017146940772
15 latent fact

In [21]:

# hyper-param config
num_iterations = 20
ranks = [10,12,15]
reg_params = [0.1,0.2,0.01,0.15,0.3]

# grid search and select best model
start_time = time.time()
final_model = train_ALS(train_df, test_df, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

10 latent factors and regularization = 0.1: validation RMSE is 2.1817911638725316
10 latent factors and regularization = 0.2: validation RMSE is 1.9870114293571355
10 latent factors and regularization = 0.01: validation RMSE is 3.271182742360547
10 latent factors and regularization = 0.15: validation RMSE is 2.056243778203823
10 latent factors and regularization = 0.3: validation RMSE is 1.9307809997008545
12 latent factors and regularization = 0.1: validation RMSE is 2.1688354940595027
12 latent factors and regularization = 0.2: validation RMSE is 1.9832647467286304
12 latent factors and regularization = 0.01: validation RMSE is 3.1723252907184367
12 latent factors and regularization = 0.15: validation RMSE is 2.0498283610823465
12 latent factors and regularization = 0.3: validation RMSE is 1.9286660606362702
15 latent factors and regularization = 0.1: validation RMSE is 2.1543379501098836
15 latent factors and regularization = 0.2: validation RMSE is 1.9794591795082017
15 latent fact

In [15]:


def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            als = ALS(maxIter=num_iters, implicitPrefs=False,\
                      userCol="user_id_numeric", itemCol="business_id_numeric", ratingCol="stars",coldStartStrategy="drop")
            model = als.fit(ratings=train_data)    # (userID, productID, rating) tuple)
            # make prediction
            valid_data = validation_data.map(lambda p: (p[0], p[1]))
            predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
            # get the rating result
            ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
            # get the RMSE
            MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
            error = math.sqrt(MSE)
            print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model



In [ ]:
import time

# hyper-param config
num_iterations = 10
ranks = [8, 10]
reg_params = [0.001, 0.01]

# grid search and select best model
start_time = time.time()
final_model = train_ALS(train_df, test_df, num_iterations, reg_params, ranks)
import time
print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))



In [ ]:
# training the model
als = ALS(maxIter=5, implicitPrefs=True,userCol="user_id_numeric", itemCol="business_id_numeric", ratingCol="stars",coldStartStrategy="drop")
model = als.fit(train_df)

In [10]:
# predict using the testing datatset
predictions = model.transform(test_df)
predictions.show()

In [ ]:
##

In [ ]:
def currentLikes(ua,ub,userId,limit):
 df = ua.join(ub,ua.artistId==ub.artistId)\
 .filter(ua.userId==userId)\
 .sort(ua.playCount.desc())\
 .select(ua.userId,ua.playCount,ub.artistName)\
 .limit(limit)
 return df# display top 10 liked artists for user 2062243
currentLikes(ua,ub,2062243,10).show(truncate=False)

# content based

In [14]:


from pyspark.ml.clustering import KMeans, KMeansModel

cols = ["business_id_numeric","business_id","city","state","name","stars","categories"]

In [69]:
business_df = spark.read.parquet("yelp_academic_dataset_business.parquet").select(cols).alias("business_df")#.limit(10)
business_df.show()#pandas_api().head()#.show(1)


+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+
|business_id_numeric|         business_id|        city|state|                name|stars|          categories|
+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+
|                  1|xKoz9eM8hUELf5qix...|  Plainfield|   IN|African Plum Home...|  4.5|Home & Garden, Ho...|
|                  2|yiE_Iym7A9wa6tyJH...|       Tampa|   FL|      Craft Fry Wing|  4.5|Cheesesteaks, Bub...|
|                  3|7d9X9wnm_35UcdJRJ...|Indianapolis|   IN|          China King|  4.0|Restaurants, Chinese|
|                  4|CFPWVAFSP3Ktfe-cE...|Philadelphia|   PA|Liberty Real Esta...|  1.5|Home Services, Re...|
|                  5|EgZH_qXEQH_WJyNFg...|   Nashville|   TN|               Jamie|  3.0|Shopping, Accesso...|
|                  6|b3KxHgc7le1Mc2Q3O...|    Edmonton|   AB|    Flowers By Merle|  3.5|Flowers & Gifts, ...|
|         

In [61]:
business_df.printSchema()

root
 |-- business_id_numeric: integer (nullable = true)
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- name: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- Diet

In [37]:
categories_vect =business_df.select(col("business_id_numeric"), F.explode(split(F.trim(col("categories")),"\\,"))).alias("categories_vect")
categories_vect = categories_vect.withColumn("col",F.trim(categories_vect.col))
categories_vect.show()


+-------------------+--------------------+
|business_id_numeric|                 col|
+-------------------+--------------------+
|                  1|       Home & Garden|
|                  1|          Home Decor|
|                  1|    Furniture Stores|
|                  1|            Shopping|
|                  2|        Cheesesteaks|
|                  2|          Bubble Tea|
|                  2|                Food|
|                  2|             Burgers|
|                  2|       Chicken Wings|
|                  2|        Coffee & Tea|
|                  2|         Restaurants|
|                  3|         Restaurants|
|                  3|             Chinese|
|                  4|       Home Services|
|                  4|Real Estate Services|
|                  4|         Real Estate|
|                  5|            Shopping|
|                  5|         Accessories|
|                  5|    Women's Clothing|
|                  5|             Fashion|
+----------

In [43]:
join_1 = business_df.join(categories_vect,categories_vect.business_id_numeric==business_df.business_id_numeric).\
drop(categories_vect.business_id_numeric).withColumn("value",lit(1))
final_df.show(2)

+--------------------+----------+--------------------+-----+--------------------+-------------------+----------------+-----+
|         business_id|      city|                name|stars|          categories|business_id_numeric|             col|value|
+--------------------+----------+--------------------+-----+--------------------+-------------------+----------------+-----+
|xKoz9eM8hUELf5qix...|Plainfield|African Plum Home...|  4.5|Home & Garden, Ho...|                  1|        Shopping|    1|
|xKoz9eM8hUELf5qix...|Plainfield|African Plum Home...|  4.5|Home & Garden, Ho...|                  1|Furniture Stores|    1|
+--------------------+----------+--------------------+-----+--------------------+-------------------+----------------+-----+
only showing top 2 rows



In [51]:
join_1.filter(col("business_id_numeric")==3794).show()

+--------------------+------------+----------+-----+--------------------+-------------------+--------------------+-----+
|         business_id|        city|      name|stars|          categories|business_id_numeric|                 col|value|
+--------------------+------------+----------+-----+--------------------+-------------------+--------------------+-----+
|R0JUjyjGEzUlY3auV...|Philadelphia|North Bowl|  4.0|Restaurants, Vega...|               3794|Event Planning & ...|    1|
|R0JUjyjGEzUlY3auV...|Philadelphia|North Bowl|  4.0|Restaurants, Vega...|               3794|           Nightlife|    1|
|R0JUjyjGEzUlY3auV...|Philadelphia|North Bowl|  4.0|Restaurants, Vega...|               3794|                Bars|    1|
|R0JUjyjGEzUlY3auV...|Philadelphia|North Bowl|  4.0|Restaurants, Vega...|               3794|             Bowling|    1|
|R0JUjyjGEzUlY3auV...|Philadelphia|North Bowl|  4.0|Restaurants, Vega...|               3794|             Lounges|    1|
|R0JUjyjGEzUlY3auV...|Philadelph

In [ ]:
df_pivot = join_1.groupBy("business_id_numeric").pivot("col").agg(F.first("value")).alias("df_pivot").fillna(0)
#df_pivot.show()

In [61]:
business_filtered_df=business_df.select("business_id","business_id_numeric","city","name")
business_final_df_2 = business_filtered_df.join(df_pivot,df_pivot.business_id_numeric==business_filtered_df.business_id_numeric).\
drop(df_pivot.business_id_numeric)
business_final_df_2.pandas_api().head(5)


,business_id,business_id_numeric,city,name,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,Active Life,Acupuncture,Addiction Medicine,Adoption Services,Adult,Adult Education,Adult Entertainment,Advertising,Aerial Fitness,Aerial Tours,Aestheticians,Afghan,African,Air Duct Cleaning,Aircraft Dealers,Aircraft Repairs,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Airsoft,Allergists,Alternative Medicine,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Anesthesiologists,Animal Assisted Therapy,Animal Physical Therapy,Animal Shelters,Antiques,Apartment Agents,Apartments,Appliances,Appliances & Repair,Appraisal Services,Aquarium Services,Aquariums,Arabic,Arcades,Archery,Architects,Architectural Tours,Argentine,Armenian,Art Classes,Art Consultants,Art Galleries,Art Installation,Art Museums,Art Restoration,Art Schools,Art Space Rentals,Art Supplies,Art Tours,Artificial Turf,Arts & Crafts,Arts & Entertainment,Asian Fusion,Assisted Living Facilities,Astrologers,Attraction Farms,Auction Houses,Audio/Visual Equipment Rental,Audiologist,Australian,Austrian,Auto Customization,Auto Detailing,Auto Glass Services,Auto Insurance,Auto Loan Providers,Auto Parts & Supplies,Auto Repair,Auto Security,Auto Upholstery,Automotive,Aviation Services,Awnings,Axe Throwing,Ayurveda,Baby Gear & Furniture,Backflow Services,Badminton,Bagels,Bail Bondsmen,Bakeries,Balloon Services,Bangladeshi,Bankruptcy Law,Banks & Credit Unions,Bar Crawl,Barbeque,Barbers,Barre Classes,Bars,Bartenders,Bartending Schools,Baseball Fields,Basketball Courts,Basque,Battery Stores,Batting Cages,Beach Bars,Beach Equipment Rentals,Beaches,Beauty & Spas,Bed & Breakfast,Beer,Beer Bar,Beer Gardens,Beer Hall,Beer Tours,Behavior Analysts,Belgian,Bespoke Clothing,Beverage Store,Bicycle Paths,Bicycles,Bike Parking,Bike Rentals,Bike Repair,Bike Repair/Maintenance,Bike Sharing,Bike Shop,Bike tours,Bikes,Billing Services,Bingo Halls,Biohazard Cleanup,Bird Shops,Bistros,Blood & Plasma Donation Centers,Blow Dry/Out Services,Boat Charters,Boat Dealers,Boat Parts & Supplies,Boat Repair,Boat Tours,Boating,Bocce Ball,Body Contouring,Body Shops,Bookbinding,Bookkeepers,Books,Bookstores,Boot Camps,Botanical Gardens,Boudoir Photography,Bounce House Rentals,Bowling,Boxing,Brasseries,Brazilian,Brazilian Jiu-jitsu,Breakfast & Brunch,Breweries,Brewing Supplies,Brewpubs,Bridal,British,Bubble Soccer,Bubble Tea,Buddhist Temples,Buffets,Building Supplies,Burgers,Burmese,Bus Rental,Bus Stations,Bus Tours,Buses,Business Consulting,Business Financing,Business Law,Butcher,CPR Classes,CSA,Cabaret,Cabinetry,Cafes,Cafeteria,Cajun/Creole,Calabrian,Calligraphy,Cambodian,Campgrounds,Canadian (New),Candle Stores,Candy Stores,Cannabis Clinics,Cannabis Collective,Cannabis Dispensaries,Cannabis Tours,Cantonese,Car Auctions,Car Brokers,Car Buyers,Car Dealers,Car Inspectors,Car Rental,Car Share Services,Car Stereo Installation,Car Wash,Car Window Tinting,Cardio Classes,Cardiologists,Cards & Stationery,Career Counseling,Caribbean,Caricatures,Carousels,Carpenters,Carpet Cleaning,Carpet Dyeing,Carpet Installation,Carpeting,Casinos,Caterers,Ceremonial Clothing,Challenge Courses,Champagne Bars,Check Cashing/Pay-day Loans,Cheerleading,Cheese Shops,Cheese Tasting Classes,Cheesesteaks,Chicken Shop,Chicken Wings,Child Care & Day Care,Childbirth Education,Childproofing,Children's Clothing,Children's Museums,Chimney Sweeps,Chinese,Chinese Martial Arts,Chiropractors,Chocolatiers & Shops,Christmas Trees,Churches,Cideries,Cigar Bars,Cinema,Circuit Training Gyms,Circus Schools,Climbing,Clock Repair,Clothing Rental,Clowns,Club Crawl,Cocktail Bars,Coffee & Tea,Coffee & Tea Supplies,Coffee Roasteries,Coffeeshops,College Counseling,Colleges & Universities,Colombian,Colonics,Comedy Clubs,Comfort Food,Comic Books,Commercial Real Estate,Commercial Truck Dealers,Commercial Truck Repair,Commissioned Artists,Community Centers,Community Gardens,Community Service/Non-

In [65]:
#business_filtered_df=business_df.select("business_id","business_id_numeric","city","name")
business_final_df = join_1.drop("value","categories").withColumnRenamed("col","categories")
business_final_df.pandas_api().head(5)


,business_id,city,name,stars,business_id_numeric,categories
0,xKoz9eM8hUELf5qixexeJQ,Plainfield,African Plum Home & More,4.5,1,Shopping
1,xKoz9eM8hUELf5qixexeJQ,Plainfield,African Plum Home & More,4.5,1,Furniture Stores
2,xKoz9eM8hUELf5qixexeJQ,Plainfield,African Plum Home & More,4.5,1,Home Decor
3,xKoz9eM8hUELf5qixexeJQ,Plainfield,African Plum Home & More,4.5,1,Home & Garden
4,yiE_Iym7A9wa6tyJHkWzhw,Tampa,Craft Fry Wing,4.5,2,Restaurants


### df correction

In [70]:
business_df.show()

+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+
|business_id_numeric|         business_id|        city|state|                name|stars|          categories|
+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+
|                  1|xKoz9eM8hUELf5qix...|  Plainfield|   IN|African Plum Home...|  4.5|Home & Garden, Ho...|
|                  2|yiE_Iym7A9wa6tyJH...|       Tampa|   FL|      Craft Fry Wing|  4.5|Cheesesteaks, Bub...|
|                  3|7d9X9wnm_35UcdJRJ...|Indianapolis|   IN|          China King|  4.0|Restaurants, Chinese|
|                  4|CFPWVAFSP3Ktfe-cE...|Philadelphia|   PA|Liberty Real Esta...|  1.5|Home Services, Re...|
|                  5|EgZH_qXEQH_WJyNFg...|   Nashville|   TN|               Jamie|  3.0|Shopping, Accesso...|
|                  6|b3KxHgc7le1Mc2Q3O...|    Edmonton|   AB|    Flowers By Merle|  3.5|Flowers & Gifts, ...|
|         

In [25]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 54.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [28]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")



[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
stop = set(stopwords.words("english"))
stop.add("&")
type(stop)

set

In [101]:

business_concat = business_df.withColumn('concat_col', 
                    F.concat(F.col("state"),F.lit(" "),F.col("city"),F.lit(" "), F.col("name"),F.lit(" "),F.col("categories")))
business_concat = business_concat.withColumn("concat_col",F.trim(col("concat_col")))
business_final_df = bussines_concat.withColumn("concat_col",
    F.regexp_replace(col("concat_col"), r',|\.|&|\\|\||-|_', ''))



In [105]:
business_final_df.show(1,truncate=False)

+-------------------+----------------------+----------+-----+------------------------+-----+-----------------------------------------------------+---------------------------------------------------------------------------------------+
|business_id_numeric|business_id           |city      |state|name                    |stars|categories                                           |concat_col                                                                             |
+-------------------+----------------------+----------+-----+------------------------+-----+-----------------------------------------------------+---------------------------------------------------------------------------------------+
|1                  |xKoz9eM8hUELf5qixexeJQ|Plainfield|IN   |African Plum Home & More|4.5  |Home & Garden, Home Decor, Furniture Stores, Shopping|IN Plainfield African Plum Home  More Home  Garden Home Decor Furniture Stores Shopping|
+-------------------+----------------------+----------+-----

In [119]:
business_final_df.count()

150346

In [118]:
business_final_df.dropna().count()

150243

### training KMeans

In [89]:

from pyspark.ml import Pipeline
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator


In [120]:
train_df, test_df = business_final_df.dropna().randomSplit([0.8,0.2])
train_df = train_df.cache()
test_df = test_df.cache()


In [107]:
train_df.printSchema()

root
 |-- business_id_numeric: integer (nullable = true)
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- categories: string (nullable = true)
 |-- concat_col: string (nullable = true)



In [121]:
tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
wordsData = tokenizer.transform(train_df)
wordsData.show(1,truncate=False)

+-------------------+----------------------+----------+-----+------------------------+-----+-----------------------------------------------------+---------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------+
|business_id_numeric|business_id           |city      |state|name                    |stars|categories                                           |concat_col                                                                             |words                                                                                                  |
+-------------------+----------------------+----------+-----+------------------------+-----+-----------------------------------------------------+---------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------

In [122]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
featurizedData.show(1)

+-------------------+--------------------+----------+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|business_id_numeric|         business_id|      city|state|                name|stars|          categories|          concat_col|               words|         rawFeatures|
+-------------------+--------------------+----------+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|                  1|xKoz9eM8hUELf5qix...|Plainfield|   IN|African Plum Home...|  4.5|Home & Garden, Ho...|IN Plainfield Afr...|[in, plainfield, ...|(20,[3,5,6,9,12,1...|
+-------------------+--------------------+----------+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [125]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("business_id_numeric", "features").show(1,truncate=False)

+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|business_id_numeric|features                                                                                                                                                                            |
+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1                  |(20,[3,5,6,9,12,16,17,18],[1.9321503062943461,1.6231472877482869,2.6335986576689865,1.1255498308708007,0.9996480163183266,0.5326419195440175,1.4711981084809076,0.8989165532996157])|
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [149]:
Kmeans = KMeans().setK(2).setSeed(42)
features_knn = rescaledData.select("features")
model = Kmeans.fit(features_knn)

In [147]:
predictions_kmeans = model.transform(rescaledData)
predictions_kmeans.show()

+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|business_id_numeric|         business_id|        city|state|                name|stars|          categories|          concat_col|               words|         rawFeatures|            features|prediction|
+-------------------+--------------------+------------+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                  1|xKoz9eM8hUELf5qix...|  Plainfield|   IN|African Plum Home...|  4.5|Home & Garden, Ho...|IN Plainfield Afr...|[in, plainfield, ...|(20,[3,5,6,9,12,1...|(20,[3,5,6,9,12,1...|         1|
|                  3|7d9X9wnm_35UcdJRJ...|Indianapolis|   IN|          China King|  4.0|Restaurants, Chinese|IN Indianapolis C...|[in, indianapolis...|(20,[0,3,14,19],[...|(20,[0,3

### IDF-KMeans PIPELINE

In [166]:
tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
Kmeans = KMeans().setK(25).setSeed(42)

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf,Kmeans])
pipeline = pipeline.fit(train_df)
predictions = pipeline.transform(test_df)
predictions.select("business_id_numeric","business_id", "features","prediction").show(1,truncate=False)


+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|business_id_numeric|features                                                                                                                                                                                                                                                                                                                                                                                             |prediction|
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------

In [73]:
to_encode = ['city','categories']
encoded = ['ohe_city','ohe_categories']
indexed = ['indx_city','indx_categories']

stringIndexer = StringIndexer(inputCols=to_encode, outputCols=indexed, handleInvalid = "keep")
encoder = OneHotEncoder(dropLast=True, inputCols=indexed, outputCols=encoded, handleInvalid = "keep" )


assembler = VectorAssembler(inputCols= encoded, outputCol="features")

Kmeans = KMeans().setK(2).setSeed(42)

pipeline = Pipeline(stages=[stringIndexer, encoder, assembler, Kmeans])

pipeline = pipeline.fit(train_df)



In [ ]:
predictions = pipeline.transform(test_df)
predictions.show(1)

In [167]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = -0.076118928543295
Cluster Centers: 
[0.40364794 0.43912408 0.48411637 0.48453413 0.56000797 0.53735476
 0.45197766 0.48632112 0.54620698 0.43447679 0.51454807 1.78743527
 0.48195794 0.50664759 0.55579789 0.46214658 0.60220821 0.56057759
 0.61121999 0.52548715]
[4.64419700e-01 4.92993136e-01 4.89581994e-01 5.02091700e-01
 4.96375990e-01 4.87562322e-01 4.48750633e-01 5.47228946e-01
 5.33678383e-01 4.76900705e-01 5.01898131e-01 2.95521653e-05
 4.28752685e-01 5.23708148e-01 5.53078702e-01 5.07080907e-01
 4.62079293e-01 4.43241465e-01 4.26748762e-01 5.26349164e-01]


In [186]:
for k in [2+ i for i in range(0,10) ]:
  
  TfIdfKMeansTrainer(k)
  

2 clusters and Silhouette with squared euclidean distance = 0.1804815344086881
3 clusters and Silhouette with squared euclidean distance = 0.15823452932701282
4 clusters and Silhouette with squared euclidean distance = 0.1928495493519425
5 clusters and Silhouette with squared euclidean distance = 0.11152592026791004
6 clusters and Silhouette with squared euclidean distance = 0.10425933109976084
7 clusters and Silhouette with squared euclidean distance = 0.10648503046037486
8 clusters and Silhouette with squared euclidean distance = 0.11080322750831334
9 clusters and Silhouette with squared euclidean distance = 0.11148030868898186
10 clusters and Silhouette with squared euclidean distance = 0.11047653253396048
11 clusters and Silhouette with squared euclidean distance = 0.11345062726434875
12 clusters and Silhouette with squared euclidean distance = 0.09563779755629293
13 clusters and Silhouette with squared euclidean distance = 0.09890642688657138
14 clusters and Silhouette with square

In [195]:

for k in [2+ i for i in range(0,20) ]:
  
  TfIdfKBisecKMeans(k)
  

2 clusters and Silhouette with squared euclidean distance = 0.05574240494874763
3 clusters and Silhouette with squared euclidean distance = 0.02727650602784901
4 clusters and Silhouette with squared euclidean distance = 0.004677607856831942
5 clusters and Silhouette with squared euclidean distance = -0.012519907220965971
6 clusters and Silhouette with squared euclidean distance = -0.008734150293493094
7 clusters and Silhouette with squared euclidean distance = -0.0205459035528675
8 clusters and Silhouette with squared euclidean distance = -0.010933211934862764
9 clusters and Silhouette with squared euclidean distance = -0.03009692281340433
10 clusters and Silhouette with squared euclidean distance = -0.027430100026643946
11 clusters and Silhouette with squared euclidean distance = -0.02201015739779191
12 clusters and Silhouette with squared euclidean distance = -0.017559828214671617
13 clusters and Silhouette with squared euclidean distance = -0.026068596153531368


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [241]:
from pyspark.ml.clustering import LDA

for k in [2+ i for i in range(0,10) ]:
  
  TF_IDF_DLA(k)

+-------------------+-------------------------------+-----------------------------------------+
|business_id_numeric|name                           |topicDistribution                        |
+-------------------+-------------------------------+-----------------------------------------+
|2                  |Craft Fry Wing                 |[0.6354711030416146,0.36452889695838536] |
|11                 |Subway                         |[0.015840559984498726,0.9841594400155013]|
|15                 |Hollywood Nails                |[0.9876957183570476,0.012304281642952452]|
|21                 |Linwood Pizza                  |[0.011327920084635736,0.9886720799153643]|
|22                 |ARC Handyman Services          |[0.5591940236674596,0.4408059763325404]  |
|24                 |Autozone                       |[0.9879197485960317,0.012080251403968262]|
|29                 |Noble Roman's Craft Pizza & Pub|[0.9936515862301054,0.00634841376989461] |
|32                 |Sacred Art Tattoo S

#### TF_IDF_KMEANS

In [ ]:
def TfIdfKMeansTrainer(k):
    
    tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    Kmeans = KMeans().setK(k).setSeed(42)

    pipeline = Pipeline(stages=[tokenizer, hashingTF, idf,Kmeans])
    pipeline = pipeline.fit(train_df)
    predictions = pipeline.transform(test_df)
    #predictions.select("business_id_numeric", "features","prediction")#.show(1,truncate=False)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()
    
    silhouette = evaluator.evaluate(predictions)
    print("{} clusters and Silhouette with squared euclidean distance = {}".format(k,silhouette))

    


#### TF_IDF_KBISECK_MEANS

In [194]:
def TfIdfKBisecKMeans(k):
    
    tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    bkm = BisectingKMeans().setK(k).setSeed(42)

    pipeline = Pipeline(stages=[tokenizer, hashingTF, idf,bkm])
    pipeline = pipeline.fit(train_df)
    predictions = pipeline.transform(test_df)
    predictions2=predictions.select("prediction")
    #predictions.select("business_id_numeric", "features","prediction")#.show(1,truncate=False)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()
    
    silhouette = evaluator.evaluate(predictions)
    print("{} clusters and Silhouette with squared euclidean distance = {}".format(k,silhouette))

    

#### TF_IDF_DLA

In [240]:
def TF_IDF_DLA(k):
    
    tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
    countVectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    lda = LDA(k=k).setSeed(42)

    pipeline = Pipeline(stages=[tokenizer, countVectorizer, idf,lda])
    pipeline = pipeline.fit(train_df)
    predictions = pipeline.transform(test_df)
    #predictions.select("business_id_numeric", "features","prediction")#.show(1,truncate=False)
    print(predictions.select("business_id_numeric","name","topicDistribution").show(10,truncate=False))
    # Evaluate clustering by computing Silhouette score
    #ll = lda.logLikelihood(train_df)
    #lp = lda.logPerplexity(train_df)
    #print("{} clusters and Silhouette with squared euclidean distance = {}".format(k,silhouette))
    #print("log likehood: {}  and log perplexity: {}".format(ll,lp))
    # Describe topics.
    topics = pipeline.stages[-1].describeTopics(3)
    terms = pipeline.stages[-3].vocabulary
    
    def indices_to_terms(indices, terms=terms):
        terms_subset = [terms[index] for index in indices]
        return terms_subset
    # Defining Spark UDF from above function
    udf_indices_to_terms = F.udf(indices_to_terms, ArrayType(StringType()))

    topics = (
       topics
       .withColumn("terms", udf_indices_to_terms(F.col("termIndices")))
    )
    print("The topics described by their top-weighted terms:")
    topics.show(truncate=False)

    print("============================================================")
    
    

In [214]:
tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
lda = LDA(k=2).setSeed(42)

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf,lda])

pipeline_metrics = Pipeline(stages=[tokenizer, hashingTF, idf])

pipeline_metrics = pipeline_metrics.fit(train_df)

pipeline = pipeline.fit(train_df)
predictions = pipeline.transform(test_df)
    #predictions.select("business_id_numeric", "features","prediction")#.show(1,truncate=False)



In [207]:
pipeline.stages

[Tokenizer_135c8dce02c3,
 HashingTF_3b1b2290e4af,
 IDFModel: uid=IDF_4394ec3d2fcc, numDocs=120249, numFeatures=262144,
 LocalLDAModel: uid=LDA_4b0714c1790b, k=2, numFeatures=262144]

In [ ]:
pipeline.stages[-1].logPerplexity(pipeline_metrics.transform(train_df))

In [215]:
pipeline.stages[-1].logLikelihood(pipeline_metrics.transform(train_df))

-49783831.75808013

In [ ]:
    # Evaluate clustering by computing Silhouette score
ll = pipeline.logLikelihood(dataset)
lp = pipeline.logPerplexity(dataset)
print("{} clusters and Silhouette with squared euclidean distance = {}".format(k,silhouette))
print("log likehood: {}  and log perplexity: {}".format(ll,lp))
# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

#### TF_IDF COSINE SIMILARITY

In [ ]:
def TF_IDF_COSINE_SIM(k):
    
    tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    

    pipeline = Pipeline(stages=[tokenizer, hashingTF, idf])
    pipeline = pipeline.fit(train_df)
    predictions = pipeline.transform(test_df)
    
    dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())
    predictions.alias("i").join(predictions.alias("j"), F.col("i.business_id_numeric") < F.col("j.business_id_numeric"))\
    .select(
        psf.col("i.ID").alias("i"), 
        psf.col("j.ID").alias("j"), 
        dot_udf("i.features", "j.features").alias("similarity"))\
    .sort("i", "j")\
    .show()

    

In [ ]:
from pyspark.ml.feature import Normalizer
tokenizer = Tokenizer(inputCol="concat_col", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="norm")
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf,normalizer])
pipeline = pipeline.fit(train_df)
predictions = pipeline.transform(train_df)#.limit(20)
    
dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())
predictions.alias("i").join(predictions.alias("j"), F.col("i.business_id_numeric") < F.col("j.business_id_numeric"))\
    .select(
        F.col("i.business_id_numeric").alias("i"), 
        F.col("j.business_id_numeric").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("similarity"))\
    .sort("i", "j")\
    .show()
